In [0]:
dbutils.widgets.text("catalog", "etl_demo")  
catalog        = dbutils.widgets.get("catalog").strip()
spark.sql(f"USE CATALOG {catalog}")
print("CATALOG:", catalog)

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS silver;

CREATE TABLE IF NOT EXISTS silver.customer_dim (
  customer_id STRING,
  name STRING,
  city STRING,
  row_hash STRING,
  valid_from DATE,
  valid_to DATE,
  is_current BOOLEAN,
  _created_time TIMESTAMP,
  _updated_time TIMESTAMP
)
USING DELTA;

-- 1️⃣ Close changed current records
MERGE INTO silver.customer_dim tgt
USING silver_staging.customer_snapshot_stg src
ON tgt.customer_id = src.customer_id
AND tgt.is_current = true
WHEN MATCHED AND tgt.row_hash <> src.row_hash THEN
  UPDATE SET
    tgt.valid_to = src.snapshot_date,
    tgt.is_current = false,
    tgt._updated_time = current_timestamp();

-- 2️⃣ Insert new / changed records
INSERT INTO silver.customer_dim
SELECT
  src.customer_id,
  src.name,
  src.city,
  src.row_hash,
  src.snapshot_date AS valid_from,
  CAST(NULL AS DATE) AS valid_to,
  true AS is_current,
  current_timestamp(),
  current_timestamp()
FROM silver_staging.customer_snapshot_stg src
LEFT ANTI JOIN silver.customer_dim tgt
  ON tgt.customer_id = src.customer_id
 AND tgt.is_current = true;
